<a href="https://colab.research.google.com/github/anabarrerar/Machine_Learning/blob/main/CatBoost/ICR_KFold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook requeriments


In [1]:
!pip install -q kaggle #kaggle

In [2]:
import os

def authenticate_kaggle(key_path):
    # Mount your Google Drive to Colab
    from google.colab import drive
    drive.mount('/content/drive')

    # Copy the Kaggle API key from your Drive to the .kaggle directory in your Drive
    !mkdir -p /root/.kaggle
    !cp '{key_path}/kaggle.json' /root/.kaggle/

    # Set the permissions for the Kaggle API key
    !chmod 600 /root/.kaggle/kaggle.json

    # Authenticate with the Kaggle API
    os.environ['KAGGLE_CONFIG_DIR'] = "/root/.kaggle"

    if os.environ.get('KAGGLE_CONFIG_DIR') == "/root/.kaggle":
        print('Authenticated with Kaggle API')

In [3]:
key_path = "/content/drive/MyDrive/Colab Notebooks"

authenticate_kaggle(key_path)

Mounted at /content/drive
Authenticated with Kaggle API


In [4]:
! kaggle competitions download -c icr-identify-age-related-conditions

  0% 0.00/150k [00:00<?, ?B/s]
100% 150k/150k [00:00<00:00, 89.1MB/s]


In [5]:
import zipfile

zip_file_path = '/content/drive/MyDrive/iterative-stratification-master.zip'
extract_path = '/content'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

## Install dependencias

In [6]:
#!pip install dataprep -q # verbose 

In [7]:
! pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.6 MB/s eta 0:00:00


In [8]:
! pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 9.1 MB/s eta 0:00:00


In [9]:
! pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.9 MB/s eta 0:00:00


## Import dependencies



In [10]:
#from dataprep.datasets import load_dataset
#from dataprep.eda import create_report

In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import os
from copy import deepcopy
from functools import partial
import random
import gc

# Import sklearn classes for model selection, cross validation, and performance evaluation
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
from category_encoders import OneHotEncoder, OrdinalEncoder, CountEncoder
from imblearn.under_sampling import RandomUnderSampler

# Import libraries for Hypertuning
import optuna

# Import libraries for gradient boosting
import xgboost as xgb
import lightgbm as lgb
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.svm import NuSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from catboost import CatBoost, CatBoostRegressor, CatBoostClassifier
from catboost import Pool


# Suppress warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

## Directory Manipulation

In [12]:
%pwd

'/content'

In [13]:
%ls

drive/                                   iterative-stratification-master/
icr-identify-age-related-conditions.zip  sample_data/


In [14]:
!unzip icr-identify-age-related-conditions.zip

Archive:  icr-identify-age-related-conditions.zip
  inflating: greeks.csv              
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [15]:
%ls

drive/                                   sample_data/
greeks.csv                               sample_submission.csv
icr-identify-age-related-conditions.zip  test.csv
iterative-stratification-master/         train.csv


In [16]:
%cd iterative-stratification-master/ 

/content/iterative-stratification-master


In [17]:
pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/iterative-stratification-master
  Preparing metadata (setup.py) ... done
  Created wheel for iterative-stratification: filename=iterative_stratification-0.1.7-py3-none-any.whl size=8413 sha256=2ece9c0dc1333dfa8e89132d414a0e4e5fec938d5d7189c7c5ab53d4f7054760
  Stored in directory: /root/.cache/pip/wheels/f8/4d/83/949cd9d1b0412b4a4416b82e747ea66dc3d86b6bc5436734de
Successfully built iterative-stratification


In [18]:
%cd ..

/content


# Description

The goal of this project is to predict if a person has one or more of any of three medical conditions (Class 1), or none of the three medical conditions (Class 0). 

To determine if someone has these medical conditions requires a long and intrusive process to collect information from patients. With predictive models, we can shorten this process and keep patient details private by collecting key characteristics relative to the conditions, then encoding these characteristics.

This project will help researchers discover the relationship between measurements of certain characteristics and potential patient conditions.



## Dataset Description

The competition data comprises over fifty anonymized health characteristics linked to three age-related conditions. Your goal is to predict whether a subject has or has not been diagnosed with one of these conditions -- a binary classification problem.



**train.csv - The training set** 617 rows, 58 columns
* `Id:` Unique identifier for each observation.
* `AB-GL:` Fifty-six anonymized health characteristics. All are numeric except for EJ, which is categorical and binomial.
* `Class:` A binary target, 1 indicates the subject has been diagnosed with one of the three conditions, 0 indicates they have not.


**test.csv - The test set** The goal is to predict the probability that a subject in this set belongs to each of the two classes.


**greeks.csv - Supplemental metadata**, only available for the training set. 617 rows, 6 columns
* `Alpha:` Identifies the type of age-related condition, if present.

    A: No age-related condition. Corresponds to class 0.

    B, D, G: The three age-related conditions. Correspond to class 1.

* `Beta, Gamma, Delta:` Three experimental characteristics.
* `Epsilon:`The date the data for this subject was collected. Note that all of the data in the test set was collected after the training set was collected.

# Load data

In [19]:
Train = pd.read_csv('train.csv')
Test = pd.read_csv('test.csv')
greeks =pd.read_csv('greeks.csv')
sample_sub=pd.read_csv('sample_submission.csv')

<class 'pandas.core.frame.DataFrame'>


## Loss Function

**logloss function**
Also known as cross-entropy loss or logistic loss, is commonly used as a loss function in binary classification problems. It measures the dissimilarity between the predicted probabilities and the true binary labels.

The **balance_logloss** is an implementation of the *balanced log loss* evaluation metric. This metric is used in unbalanced binary classification problems, where the classes have different numbers of samples.

In [26]:
def balance_logloss(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-15, 1-1e-15) # avoid values ​​close to zero or one
    y_pred / np.sum(y_pred, axis=1)[:, None] # normalize the predictions
    nc = np.bincount(y_true) # count the number of samples of each class in the vector of true labels
    w0, w1 = 1/(nc[0]/y_true.shape[0]), 1/(nc[1]/y_true.shape[0]) # the weights w0, w1 compensate the class imbalance. 
    logloss = (-w0/nc[0]*(np.sum(np.where(y_true==0,1,0) * np.log(y_pred[:,0]))) - w1/nc[1]*(np.sum(np.where(y_true!=0,1,0) * np.log(y_pred[:,1])))) / (w0+w1)
    
    return logloss

## Model

**Creating folds for Cross Validation**

In [20]:
greeks_copy=greeks.copy()
greeks = pd.merge(Train[['Id', 'Class']], greeks, on='Id')

<class 'pandas.core.frame.DataFrame'>


### Stratified k-fold cross-validation

Overall, this code performs multilabel stratified k-fold cross-validation by dividing the data into `n_splits` folds and assigning a fold index to each row in the "kfold" column of the DataFrame. The purpose of this approach is to ensure that each fold represents a balanced distribution of the target variables `(greeks.iloc[:,1:-1])`, suitable for training and evaluating a multilabel classification model.

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

Train['kfold'] = -1

kf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_indicies, valid_indicies) in enumerate(kf.split(X=Train, y=greeks.iloc[:,1:-1])):
    Train.loc[valid_indicies, "kfold"] = fold

In [ ]:
Train.to_csv("train_folds.csv", index=False) #Export the csv file
#Train=pd.read_csv("train_folds.csv")

### Catboost

In [ ]:
Train['EJ'] = Train['EJ'].replace({'A': 0, 'B': 1})
Test['EJ']  = Test['EJ'].replace({'A': 0, 'B': 1})

In [ ]:
features = Train.columns[1:-2]
label    = Train.columns[-2]

In [27]:
final_valid_predictions = {}
final_test_predictions = []
s  = []
bs = []

for k in range(5): #Iterate through 5 folds
    print('------------------ Fold: '+str(k)) #Print number of iteration
    train     = Train[Train['kfold'] !=k].reset_index(drop=True) #training set, except current kfold
    val       = Train[Train['kfold'] ==k].reset_index(drop=True) #validation set, current kfold
    valid_ids = val.Id.values.tolist() #Unique identifiers in the validation set (val.Id) are obtained and stored in the valid_ids list.


    #train_dataset and eval_dataset created using the training and validation data,features, and label
    train_dataset = Pool(data=train[features], label=train[label], cat_features=["EJ"] )
    eval_dataset  = Pool(data=val[features], label=val[label], cat_features=["EJ"])


    #The parameters of the CatBoostClassifier model are defined
    params = {
       "iterations": 10000,
        "verbose": 1000,
        "learning_rate": 0.003,
        "depth": 4,
        'auto_class_weights':'Balanced',
        'loss_function':'MultiClass',
        'eval_metric':'MultiClass:use_weights=False',
    }

    #An instance of the CatBoostClassifier model is created with the specified parameters.
    model = CatBoostClassifier(**params)

    #The model is trained on the training set.
    model.fit(train_dataset, eval_set=eval_dataset, use_best_model=True)

    #Probability predictions are generated for the validation set and the test set.
    preds_valid = model.predict_proba(val[features])
    preds_test  = model.predict_proba(Test[features])

    final_test_predictions.append(preds_test) #Append test set predictions
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid))) #Validation set predictions added to a dictionary, sample identifiers as keys. 
    logloss  = log_loss(val[label], preds_valid) #Calculate the log-loss metric 
    blogloss = balance_logloss(val[label], preds_valid) #Calculate the balanced log-loss metric 

    s.append(logloss) #Append logloss values
    bs.append(blogloss) #Append balanced logloss values

    print(k, logloss, blogloss)
print('Log loss:')
print(s)
print(np.mean(s), np.std(s))  
print('Balance Log loss:')
print(bs)
print(np.mean(bs), np.std(bs))

------------------ Fold: 0
0:	learn: 0.6918613	test: 0.6918524	best: 0.6918524 (0)	total: 60.4ms	remaining: 10m 4s
1000:	learn: 0.3112258	test: 0.3129799	best: 0.3129799 (1000)	total: 12.4s	remaining: 1m 51s
2000:	learn: 0.2102446	test: 0.2228566	best: 0.2228566 (2000)	total: 28s	remaining: 1m 52s
3000:	learn: 0.1412684	test: 0.1690151	best: 0.1690151 (3000)	total: 36.5s	remaining: 1m 25s
4000:	learn: 0.0976817	test: 0.1384604	best: 0.1384604 (4000)	total: 43s	remaining: 1m 4s
5000:	learn: 0.0725291	test: 0.1201606	best: 0.1201606 (5000)	total: 51.3s	remaining: 51.3s
6000:	learn: 0.0561581	test: 0.1080580	best: 0.1080580 (6000)	total: 59.4s	remaining: 39.6s
7000:	learn: 0.0447400	test: 0.0994511	best: 0.0994511 (7000)	total: 1m 12s	remaining: 30.9s
8000:	learn: 0.0365569	test: 0.0929959	best: 0.0929959 (8000)	total: 1m 20s	remaining: 20.1s
9000:	learn: 0.0304336	test: 0.0879040	best: 0.0879040 (9000)	total: 1m 26s	remaining: 9.63s
9999:	learn: 0.0258011	test: 0.0839598	best: 0.0839433 

In [28]:
#Predictions of the validation set
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index() #dictionary to dataframe
final_valid_predictions.columns = ['Id', 'class_0', 'class_1'] #Column names

#Calculating the average prediction for each sample in the test set (taking the 5 folds).
final_test_predictions = (final_test_predictions[0] + final_test_predictions[1] + final_test_predictions[2] + final_test_predictions[3] + final_test_predictions[4])/5
test_dict = {}
test_dict.update(dict(zip(Test.Id.values.tolist(), final_test_predictions))) #Mapping each sample ID from the Test dataset to its corresponding average prediction from final_test_predictions. 
submission = pd.DataFrame.from_dict(test_dict, orient="index").reset_index() #Dictionary to dataframe
submission.columns = ['Id', 'class_0', 'class_1'] #Column names               

# submission.to_csv(r"submission.csv", index=False)

In [29]:
submission.head()

,Id,class_0,class_1
0,00eed32682bb,0.803432,0.196568
1,010ebe33f668,0.803432,0.196568
2,02fa521e1838,0.803432,0.196568
3,040e15f562a2,0.803432,0.196568
4,046e85c7cc7f,0.803432,0.196568


In [30]:
submission1=submission.copy() #wihout preprocessing

In [33]:
#submission1.to_csv(r"submission.csv", index=False)